## 准备数据：从文本中构建词向量

### 分词：切分文本成词

In [1]:
mySent = "This book is the best book on Python or M.L. I have ever laid eyes upon."
mySent.split()

['This',
 'book',
 'is',
 'the',
 'best',
 'book',
 'on',
 'Python',
 'or',
 'M.L.',
 'I',
 'have',
 'ever',
 'laid',
 'eyes',
 'upon.']

In [2]:
import re
# 切分除单词、数字外的任意字符串
regEx = re.compile('\\W+')
listOfTokens = regEx.split(mySent)
listOfTokens[:10]

['This', 'book', 'is', 'the', 'best', 'book', 'on', 'Python', 'or', 'M']

In [3]:
print([tok for tok in listOfTokens if len(tok) > 0][:7])
print([tok.lower() for tok in listOfTokens if len(tok) > 0][:7])

['This', 'book', 'is', 'the', 'best', 'book', 'on']
['this', 'book', 'is', 'the', 'best', 'book', 'on']


In [4]:
# emailText = open('email/ham/6.txt').read()

# Above way of calling failed because of the Windows-1252 encode
# Here is a better one
with open('email/ham/6.txt', encoding='cp1252')\
        as datafile:
    emailText = datafile.read()
emailText[:10]

'Hello,\n\nSi'

In [5]:
listOfTokens = regEx.split(emailText)
len(listOfTokens)

257

### 生成词汇表

In [6]:
# 使用函数createVocabList完成切分
import bayes
listOPost, listClasses = bayes.loadDataSet()
myVocabList = bayes.createVocabList(listOPost)
myVocabList[:10]

['steak', 'love', 'my', 'quit', 'dog', 'cute', 'ate', 'I', 'food', 'problems']

### 生成词向量

In [7]:
print(bayes.bagOfWords2Vec(myVocabList, listOPost[0])[:10])
print(bayes.bagOfWords2Vec(myVocabList, listOPost[3])[:10])

[0, 0, 1, 0, 1, 0, 0, 0, 0, 1]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 训练算法-从词向量计算概率

In [8]:
from numpy import *
listOPost, listClassesOPost = bayes.loadDataSet()
trainMat = []
for postinDoc in listOPost:
    trainMat.append(bayes.bagOfWords2Vec(myVocabList,
                                        postinDoc))
p0V, p1V, pAb = bayes.train(trainMat, listClasses)
print(p0V[:5])
print(p1V[:5])
print(pAb)

[-2.56494936 -2.56494936 -1.87180218 -3.25809654 -2.56494936]
[-3.04452244 -3.04452244 -3.04452244 -2.35137526 -1.94591015]
0.5


## 测试过程-根据现实情况修改分类器

为了与习题（3）作比较，这里修改了源码，使用相同的训练集与测试集进行训练与测试

In [6]:
# bayes.spamTest()
import bayesEdit
train_X, train_Y, test_X, test_Y = bayesEdit.spamTest()

Classification error  1 ['just', '750', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'just', 'service', 'service', 'just']
The error rate is  0.1


## 操作练习

### 实验中如何解决零概率问题

这里采用拉普拉斯平滑，相关代码为bayes.py的44行

In [ ]:
# bayes.py line 44
    p0Num=ones(numWords);p1Num=ones(numWords)

### 如何解决概率值太小会产生溢出问题

这里没有直接算出概率值，而是使用对数似然$\log P(x|c)$来进行等价与$P(x|c)$的计算，将连乘转化为连加后，不会发生太小溢出，相关代码为

In [ ]:
# bayes.py line 53,54
    p1Vec=log(p1Num/p1Denom)
    p0Vec=log(p0Num/p0Denom)
# bayes.py line 59,60
    p1=sum(vec2classfy*p1Vec)+log(pClass1)
    p0=sum(vec2classfy*p0Vec)+log(1-pClass1)

### 利用sklearn中不同的NB分类器分类该数据集

查看sklearn文档，我们发现关于Naive Bayes的分类器有四种：
* Gaussian Naive Bayes
* Multinomial Naive Bayes
* Complement Naive Bayes(Not supported in sklearn 0.0)
* Bernoulli Naive Bayes

#### Gaussian Naive Bayes

0.1

In [23]:
from sklearn.naive_bayes import MultinomialNB
mnb = MultinomialNB()

mnb.fit(train_X, train_Y)
round(1.0 - mnb.score(test_X, test_Y), 1)

0.1

In [26]:
from sklearn.naive_bayes import BernoulliNB
bnb = BernoulliNB()

bnb.fit(train_X, train_Y)
1.0 - bnb.score(test_X, test_Y)

0.4

可以看到，对于生成的某一组训练集&数据集，使用Gaussian，Multinomial或者直接的均匀先验的朴素贝叶斯的测试集准确率达到90%，而使用Bernoulli先验的朴素贝叶斯分类器测试集的准确率只有0.4。尽管由于数据量太小，结果的偏差十分巨大，但是我们还是能看到，bernoulli这种把属性直接归为出现/不出现十分粗糙，它忽略了词频信息，计算上不是正确的，分类结果十分差，而其他模型表现还算可以。

## 疑惑

在bayes.py中，函数*train()*中计算每个类所有单词出现总的次数p0Num初始化为2.0，这里我猜测是因为对于词$word_i$，它在一封email中可能出现的次数为任意自然数，所以使用laplacian correction时，不能取无穷，但是通常来说，最多的情况是，$word_i$出现的时候，次数大多都是1，所以分母中的校子取为2。